In [72]:
from flask import Flask
from flask import request
import datetime
from dateutil.parser import parse
import json
import numpy as np
import random 
import collections
import copy 
import math

In [4]:
info_pr = {'process_info': [[[[1, 1, 10], [2, 1, 11]], [[3, 1, 12]], [[2, 1, 13]]],
                            [[[3,1,14], [1,1,15], [2,1,16]], [[2,1,17], [3,1,18]], [[3,1,19]]],
                            [[[1,1,1],[2,1,2]]]],
           "job_nb": 3, 'total_op_nb': 7, 'machine_nb': 3,'machine_list':[1,2,3], 'machine_aval':[0,0,0],
           'start_date':"2022-11-01 00:00:00",
           'job_dict':{1:['order 1', 1], 2:['order 2', 1],3:['order 3', 1]},
           'machine_dict':{1:'machine 1',2:'machine 2',3:'machine 3'}
           }

app = Flask(__name__)


In [96]:
required_data = {
    "periodStartDate": "2022-11-01 00:00:00", ##! 当前排产周期的开始时间
    "orders": [ ##! 当前排产周期订单信息
        {   
            'orderID': '001',
            'productID': '0213配油盘',
            'productNB': 1
        },
        {
            'orderID': '002',
            'productID': '12M26机体',
            'productNB': 1
        },
        {
            'orderID': '003',
            'productID': 'PSI机体',
            'productNB': 2
        }
    ],
    "process": { ##! 产品加工工艺信息
        '0213配油盘': [
            {
                'machine': ['MC6000-7'],
                'processTime': [20],
                'fixture': ['F1'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [35],
                'fixture': ['F1'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [30],
                'fixture': ['F2'],
                'isInspetion': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [30],
                'fixture': ['F2'],
                'isInspetion': False
            }
        ],
        '12M26机体': [
            {
                'machine': ['MC6000-6'],
                'processTime': [50],
                'fixture': ['F3'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [75],
                'fixture': ['F4'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-6'],
                'processTime': [60],
                'fixture': ['F4'],
                'isInsepction': False
            },
            {
                'machine': ['MC6000-6'],
                'processTime': [30],
                'fixture': ['F5'],
                'isInspection': True
            }
        ],
        'PSI机体': [
            {
                'machine': ['德玛吉五轴', '乌尼恩-1#'],
                'processTime': [276, 276],
                'fixture': ['F6', 'F7'],
                'isInspection': False
            },
            {
                'machine': ['德玛吉五轴', '英赛-1#'],
                'processTime': [360, 360],
                'fixture': ['F8', 'F9'],
                'isInspection': False
            },
            {
                'machine': ['德玛吉五轴', '英赛-1#', 'BW'],
                'processTime': [276, 276, 300],
                'fixture': ['F10', 'F11', 'F12'],
                'isInspection': False
            },
            {
                'machine': ['南MTM清洗机'],
                'processTime': [252],
                'fixture': [],
                'isInspection': False
            }
        ]
    },
    "machines": ['MC6000-6', 'MC6000-7', '德玛吉五轴', '乌尼恩-1#', '英赛-1#', 'BW', '南MTM清洗机'], ##! 当前排产周期的设备列表
    "stuff": ['Jack', 'Tom', 'Mary', 'Lily', 'Bob', 'Alice', 'David', 'Eve', 'Frank', 'Grace', 'Helen', 'Ivan', 'John', 'Kevin', 'Lucy', 'Mike', 'Nancy', 'Oscar', 'Peter', 'Queen', 'Robin', 'Steve', 'Tony', 'Uma', 'Victor', 'Wendy', 'Xavier', 'Yvonne', 'Zoe'], ##! 当前排产周期的员工列表
    "machineAval": [0, 0, 0, 0, 0, 0, 0], ##! 当前排产周期的设备可用时间
    "machineNotAvalPeriod": { ##! 当前排产周期的设备不可用时间段
        'MC6000-6': [['2022-11-01 19:00:00', '2022-11-02 08:00:00'], ['2022-11-02 19:00:00', '2022-11-03 08:00:00']], ##! 夜班不可用
        'MC6000-7': [['2022-11-01 19:00:00', '2022-11-02 08:00:00'], ['2022-11-02 19:00:00', '2022-11-03 08:00:00']],
        '德玛吉五轴': [],
        '乌尼恩-1#': [],
        '英赛-1#': [],
        'BW': [],
        '南MTM清洗机': [['2022-11-01 12:00:00', '2022-11-01 18:00:00']] ##! 维修时间
    },
    "nonProcessingTime": 15, ##! 上下料、换夹具等非加工时间
}

In [97]:
def time_transform(start_time: str, period: list): 
    start_time = datetime.datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
    target = []
    if bool(period):
        for i in period:
            date1 = parse(i[0])
            date2 = parse(i[1])
            result1 = date1 - start_time
            result2 = date2 - start_time
            target.append([int(result1.total_seconds()//60), int(result2.total_seconds()//60)])
    return target

In [98]:
def info_pr_with_json(data_json):
    global info_pr 
    info_pr['machine_nb'] = len(data_json['machines'])
    dict_machine = {}
    info_pr['machine_dict'] = {}
    for index, machine_name in enumerate(data_json['machines']):
        dict_machine[machine_name] = index + 1
        info_pr['machine_dict'][index+1] = machine_name
    info_pr['machine_list'] = [i for i in range(1, info_pr['machine_nb'] + 1)]
    
    job_nb = 0
    for order in data_json['orders']:
        job_nb += order['productNB']
    info_pr['job_nb'] = job_nb
    info_pr['start_date'] = data_json['periodStartDate']
    info_pr['machineNotAvalPeriod'] = []
    for machine, machine_index in dict_machine.items():
        info_pr['machineNotAvalPeriod'].append(time_transform(data_json['periodStartDate'], data_json['machineNotAvalPeriod'][machine]))
    
    info_pr['process_info'] = []
    info_pr['total_op_nb'] = 0
    job_count = 1
    for order in data_json['orders']:
        for job_index_in_order in range(order['productNB']):
            info_pr['job_dict'][job_count] = [order['orderID'] , job_index_in_order + 1]
            job_count += 1
            job_info = []
            processes = data_json['process'][order['productID']]
            info_pr['total_op_nb'] += len(processes)
            for process in processes:
                process_info = []
                for index, machine in enumerate(process['machine']):
                    temp = [0,1,0]
                    # 设备index 赋值
                    temp[0] = dict_machine[machine]
                    ##! 暂时忽略夹具
                    # 加工时间赋值
                    temp[2] = process['processTime'][index]
                    process_info.append(temp)
                job_info.append(process_info)
            info_pr['process_info'].append(job_info)
    info_pr['machine_aval'] = data_json['machineAval']
    info_pr['nonProcessingTime'] = data_json['nonProcessingTime']
    return info_pr
    

In [99]:
res = info_pr_with_json(required_data)
print(res['job_nb'])
print(res['machine_dict'])
print(res['total_op_nb'])


4
{1: 'MC6000-6', 2: 'MC6000-7', 3: '德玛吉五轴', 4: '乌尼恩-1#', 5: '英赛-1#', 6: 'BW', 7: '南MTM清洗机'}
16


In [104]:
JOB = [i for i in range(1, res['job_nb'] + 1)]
OPERATION = {}
for i in JOB:
    OPERATION[i] = [i for i in range(1, 1+ len(res['process_info'][i-1]))]
MACHINE = {}
pt = {}
PT = {}
for i in JOB:
    for j in OPERATION[i]:
        MACHINE[(i,j)] = [res['process_info'][i-1][j-1][k][0] for k in range(len(res['process_info'][i-1][j-1]))]
        pt[(i,j)] = [res['process_info'][i-1][j-1][k][2] for k in range(len(res['process_info'][i-1][j-1]))]
        dic_ref = dict(zip(MACHINE[(i,j)], pt[(i,j)]))
        for k in MACHINE[(i,j)]:
            PT[(i,j,k)] = dic_ref[k]
INDEX_DIC = {}
lengh = 0
for i in JOB:
    for j in OPERATION[i]:
        INDEX_DIC[(i,j)] = lengh + OPERATION[i].index(j) + 1
    lengh += len(OPERATION[i])



{(1, 1): 1,
 (1, 2): 2,
 (1, 3): 3,
 (1, 4): 4,
 (2, 1): 5,
 (2, 2): 6,
 (2, 3): 7,
 (2, 4): 8,
 (3, 1): 9,
 (3, 2): 10,
 (3, 3): 11,
 (3, 4): 12,
 (4, 1): 13,
 (4, 2): 14,
 (4, 3): 15,
 (4, 4): 16}

In [106]:
OS = [1,1,1,1,2,2,4,4,2,2,3,3,4,3,3,4]
MA = [2,2,2,2,1,2,2,2,3,3,6,7,4,3,5,7]
machine_num = res['machine_nb']

def get_OS_index():
    OS_copy = OS.copy()
    OS_index = [] ##! OS中各工序对应在MA中的位置
    OS_index_PT = [] ##! OS中各工序对应该产品的第几道工序
    L = len(OS)
    for i in range(L):
        count = collections.Counter(OS_copy)
        operation_index  = len(OPERATION[OS[i]]) - count[OS[i]]
        OS_index.append(INDEX_DIC[(OS[i], OPERATION[OS[i]][operation_index])])
        OS_index_PT.append(OPERATION[OS[i]][operation_index])
        OS_copy.pop(0)
    return OS_index, OS_index_PT



[1, 2, 3, 4, 5, 6, 13, 14, 7, 8, 9, 10, 15, 11, 12, 16]
[1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 1, 2, 3, 3, 4, 4]


In [90]:
OS = [1,1,1,1,2,2,4,4,2,2,3,3,4,3,3,4]
MA = [2,2,2,2,1,2,2,2,3,3,6,7,4,3,5,7]
machine_num = res['machine_nb']

# def get_OS_index():
#     OS_copy = OS.copy()
#     OS_index = [] ##! 给MA用
#     OS_index_PT = [] ##! 给PT用
#     L = len(OS)
#     for i in range(L):
#         count = collections.Counter(OS_copy)
    
count = collections.Counter(OS)
count

Counter({1: 4, 2: 4, 4: 4, 3: 4})